In [ ]:
import itertools

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
galaxy = lines.copy()

part 1

In [ ]:
def string_insert(string, index, symbol):
    return string[:index] + symbol + string[index:]


def expand_cols(galaxy):
    cols_to_add = []
    for i in range(0, len(galaxy[0])):
        col_fields = []
        for j in range(0, len(galaxy)):
            # this is very uneconomic, but still fast enough
            col_fields.append(galaxy[j][i])
        if all([p == "." for p in col_fields]):
            cols_to_add.append(i)

    new_galaxy = []
    for r in galaxy:
        expanded_row = r
        # shift indexes by 1 for each added col
        for i in range(0, len(cols_to_add)):
            expanded_row = string_insert(expanded_row, cols_to_add[i] + i, ".")
        new_galaxy.append(expanded_row)
    return new_galaxy


# expand_cols(galaxy)

In [ ]:
def expand_rows(galaxy):
    galaxy = galaxy.copy()
    empty_row = "." * len(galaxy[0])
    i = 0
    while i < len(galaxy):
        if all([p == "." for p in galaxy[i]]):
            galaxy.insert(i, empty_row)
            # skip newly inserted row
            i += 1
        i += 1
    return galaxy


# expand_rows(galaxy)

In [ ]:
def expand_galaxy(galaxy):
    expanded_galaxy = expand_cols(galaxy)
    expanded_galaxy = expand_rows(expanded_galaxy)
    return expanded_galaxy


# expand_galaxy(galaxy)

In [ ]:
galaxy_e = expand_galaxy(galaxy)

In [ ]:
galaxies = []
for i in range(0, len(galaxy_e)):
    for j in range(0, len(galaxy_e[0])):
        if galaxy_e[i][j] == "#":
            galaxies.append((i, j))
# galaxies

In [ ]:
galaxy_combinations = list(itertools.combinations(galaxies, 2))
galaxy_dists = []
for gc in galaxy_combinations:
    dist = abs(gc[0][0] - gc[1][0]) + abs(gc[0][1] - gc[1][1])
    galaxy_dists.append(dist)
sum(galaxy_dists)

part 2

In [ ]:
# first naive brute force increasing of universe

In [ ]:
def expand_cols_m(galaxy, m):
    cols_to_add = []
    for i in range(0, len(galaxy[0])):
        col_fields = []
        for j in range(0, len(galaxy)):
            # this is very uneconomic, but still fast enough
            col_fields.append(galaxy[j][i])
        if all([p == "." for p in col_fields]):
            cols_to_add.append(i)

    new_galaxy = []
    # print(cols_to_add)
    for r in galaxy:
        expanded_row = r
        # print(r)
        # shift indexes by 1 for each added col
        for i in range(0, len(cols_to_add)):
            for t in range(0, m - 1):
                k = i * (m - 1)
                # print(f"Added at index {cols_to_add[i]+k}")
                expanded_row = string_insert(expanded_row, cols_to_add[i] + k, ".")

        new_galaxy.append(expanded_row)

    return new_galaxy


def expand_rows_m(galaxy, m):
    galaxy = galaxy.copy()
    empty_row = "." * len(galaxy[0])
    i = 0
    while i < len(galaxy):
        if all([p == "." for p in galaxy[i]]):
            for t in range(0, m - 1):
                galaxy.insert(i, empty_row)
                # skip newly inserted row
                i += 1
        i += 1
    return galaxy


def expand_galaxy_m(galaxy, m):
    expanded_galaxy = expand_cols_m(galaxy, m)
    expanded_galaxy = expand_rows_m(expanded_galaxy, m)
    return expanded_galaxy


galaxy_e = expand_galaxy_m(galaxy, 5)

In [ ]:
# Brute force was once again not working out
# new idea, see what distances change, when galaxy expands by 1
# Compute the increase of distance for each galaxy combination in that 1 step expansion
# and then scale that by the number of increases for all galaxy distances

In [ ]:
def get_distances(galaxies, m):
    galaxy_e = expand_galaxy_m(galaxy, m)
    galaxies = []
    # give id to galaxy, they are stable because traversal is stable
    # this will be used to re-identify them because coordinates change after expansion
    galaxy_id = 0
    for i in range(0, len(galaxy_e)):
        for j in range(0, len(galaxy_e[0])):
            if galaxy_e[i][j] == "#":
                galaxies.append((galaxy_id, (i, j)))
                galaxy_id += 1

    galaxy_combinations = list(itertools.combinations(galaxies, 2))
    galaxy_dists = {}
    for gc in galaxy_combinations:
        dist = abs(gc[0][1][0] - gc[1][1][0]) + abs(gc[0][1][1] - gc[1][1][1])
        galaxy_dists[(gc[0][0], gc[1][0])] = dist
    return galaxy_dists

In [ ]:
# 1 means not increased at all
d1 = get_distances(galaxies, 1)
# 2 means doubled, so a single increase per empty row or col
d2 = get_distances(galaxies, 2)

In [ ]:
increase_counts = {}
for galaxy_comb in d1.keys():
    # maximum increase would be if everything in between to galaxies was empty, so bound by size of galaxy board
    for i in range(0, max(len(galaxy), len(galaxy[0]))):
        if d1[galaxy_comb] + i == d2[galaxy_comb]:
            if i not in increase_counts:
                increase_counts[i] = 1
            else:
                increase_counts[i] += 1


print(increase_counts)

In [ ]:
# base is distances without any increase
# scale them with their respective increase values

# if there are 500 galaxies where the distance increased by 5 for expanding once,
# then that distance per galaxy will increase by 10 if expanding twice etc
# so multiply all of that and add to base distances
# quick sanity check: scale 2 should match part a
base = sum(d1.values())
scale_number = 1000000
for k, v in increase_counts.items():
    base += k * v * (scale_number - 1)
base